In [1]:
import pandas as pd
import re
import spacy
from spacy.util import minibatch, compounding
import random
from spacy import displacy
spacy.prefer_gpu()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
df = pd.read_json (r'/content/drive/MyDrive/Entity Recognition in Resumes.json',lines=True)
df

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN
...,...,...,...
215,"Mansi Thanki\nStudent\n\nJamnagar, Gujarat - E...","[{'label': ['College Name'], 'points': [{'star...",NaN
216,Anil Kumar\nMicrosoft Azure (Basic Management)...,"[{'label': ['Location'], 'points': [{'start': ...",NaN
217,Siddharth Choudhary\nMicrosoft Office Suite - ...,"[{'label': ['Skills'], 'points': [{'start': 78...",NaN
218,Valarmathi Dhandapani\nInvestment Banking Oper...,"[{'label': ['Skills'], 'points': [{'start': 92...",NaN


# Подготовка данных для Spacy

In [4]:
entity_dict = {
    'Name': "PERSON", 
    'Companies worked at': "ORGANIZATION",
    'Skills': "SKILLS",
    'Location': "LOCATION",
    'Graduation Year': 'DATE',
    'Years of Experience': 'DATE',
}

In [5]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [6]:
def get_entities(df):
    
    entities = []
    
    for i in range(len(df)):
        entity = []
    
        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass
    
        entity = mergeIntervals(entity)
        entities.append(entity)
    
    return entities

In [7]:
df['entities'] = get_entities(df)
df.head()

,content,annotation,extras,entities
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN,"[(0, 12, PERSON), (49, 58, ORGANIZATION), (60,..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN,"[(0, 14, PERSON), (62, 68, LOCATION), (625, 63..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN,"[(0, 21, PERSON), (22, 31, LOCATION), (764, 77..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN,"[(0, 12, PERSON), (54, 60, ORGANIZATION), (62,..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN,"[(0, 13, PERSON), (24, 41, ORGANIZATION), (43,..."


In [8]:
df = df.drop(['extras'], axis=1)
df = df.drop(['annotation'], axis=1)
df.head()

,content,entities
0,Abhishek Jha\nApplication Development Associat...,"[(0, 12, PERSON), (49, 58, ORGANIZATION), (60,..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[(0, 14, PERSON), (62, 68, LOCATION), (625, 63..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[(0, 21, PERSON), (22, 31, LOCATION), (764, 77..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[(0, 12, PERSON), (54, 60, ORGANIZATION), (62,..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[(0, 13, PERSON), (24, 41, ORGANIZATION), (43,..."


In [9]:
TRAIN_DATA =[]
for col in df.iloc:
  TRAIN_DATA.append((col['content'], {'entities': col['entities']}))

In [10]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

TRAIN_DATA = trim_entity_spans(TRAIN_DATA)

#Обучение

In [11]:
pip install spacy

In [12]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [13]:
nlp = create_blank_nlp(TRAIN_DATA)
optimizer = nlp.begin_training()
for i in range(10):
    losses = {}
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd = optimizer, drop=0.1, losses=losses)
    print(f"Эпоха {i}, ошибка: {losses}")

Эпоха 0, ошибка: {'ner': 30090.95131472824}
Эпоха 1, ошибка: {'ner': 14024.250839699691}
Эпоха 2, ошибка: {'ner': 9460.907696839828}
Эпоха 3, ошибка: {'ner': 9174.933850395106}
Эпоха 4, ошибка: {'ner': 14237.91988391488}
Эпоха 5, ошибка: {'ner': 8048.455585885071}
Эпоха 6, ошибка: {'ner': 10524.437786284163}
Эпоха 7, ошибка: {'ner': 13363.175489177158}
Эпоха 8, ошибка: {'ner': 7063.032096710016}
Эпоха 9, ошибка: {'ner': 9237.514056494154}


In [14]:
doc = nlp("Bob Ross lived in Florida knows Python, C#, Java and wants 1000 dollars, he worked in Google")
displacy.render(doc, style="ent", jupyter=True)

In [15]:
nlp.to_disk("NLP_model")

In [16]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Bob Ross lived in Florida knows Python, C#, Java and wants 1000 dollars, he worked in Google")
displacy.render(doc, style="ent", jupyter=True)